# # 모듈 및 함수
---

In [ ]:
import os
import glob
import re
import json
import shutil
import numpy as np
import pandas as pd

# # data processing
---

## # gt
---

### # evaluate_tracking.seqmap.training
---

In [ ]:
# yolov5, pvrcnn inference 완료한 데이터

with open('/data/NIA50/50-2/data/NIA50/학습용데이터_pre4/Suwon/ImageSets/test.txt', 'r') as f:
    scenes = [re.sub('\n', '', i)[:-5] for i in f.readlines()]
    scenes = sorted(list(set(scenes)))

In [6]:
# deepfusionmot 완료된 것만

scenes = sorted(os.listdir('/data/NIA50/50-2/data/NIA50/train_1st/trackeval/trackers/nia50_train1st'))
scenes = [i[:-4] for i in scenes]

In [ ]:
save_path = '/data/NIA50/50-2/data/NIA50/train_1st/trackeval/gt/'
os.makedirs(save_path, exist_ok=True)

seqmap = [f'{scene[:-4]} empty 0000 0011' for scene in scenes]

with open(save_path+'evaluate_tracking.seqmap.training', 'w') as f:
    f.write('\n'.join(seqmap))

### # label_02
---

In [ ]:
# yolov5, pvrcnn inference 완료한 데이터

with open('/data/NIA50/50-2/data/NIA50/학습용데이터_pre4/Suwon/ImageSets/test.txt', 'r') as f:
    scenes = [re.sub('\n', '', i)[:-5] for i in f.readlines()]
    scenes = sorted(list(set(scenes)))

In [33]:
save_path = '/data/NIA50/50-2/data/NIA50/train_1st/trackeval/gt/label_02/'
os.makedirs(save_path, exist_ok=True)

rename = {'Adult': 'Adult', 
          'Bus': 'Car',
          'Car': 'Car',
          'Kid': 'Kid',
          'Kickboard': 'Two_Wheeler',
          'Large_Truck': 'Car',
          'Light_Car': 'Car',
          'Medium_Truck': 'Car',
          'Mini_Bus': 'Car',
          'SUV': 'SUV',
          'Small_Car': 'Car',
          'Small_Truck': 'Car',
          'Special_Vehicle': 'Car',
          'Two_Wheeler': 'Two_Wheeler',
          'Van': 'Van'}


source_path = '/data/NIA50/50-2/data/NIA50/train_1st/raw/'
for scene in scenes:
    calib = source_path + scene + '/calib/camera/camera_0.json'
    with open(calib, 'r') as f:
        calib = json.load(f)
    intrinsic = np.asarray(calib['intrinsic']).reshape(3, 3)
    intrinsic = np.hstack([intrinsic, np.asarray([[0, 0, 0]]).reshape(-1, 1)])
    extrinsic = np.asarray(calib['extrinsic']).reshape(4, 4)
    
    labels = sorted(glob.glob(f'{source_path}/{scene}/label/[0-9]*.json'))
    # scene_df = pd.DataFrame(columns=['frame', 'track_id', 'type', 'truncated', 'occluded', 'alpha', 'bbox', 'dimensions', 'location', 'rotation_y'])
    frame = []
    track_id = []
    type_ = []
    # truncated = []
    # occluded = []
    # alpha = []
    # bbox = []
    # dimensions = []
    # location = []
    x1, y1, x2, y2 = [], [], [], []
    h_, w_, l_ = [], [], []
    x_, y_, z_ = [], [], []
    rotation_y = []
    for i, label in enumerate(labels):
        with open(label, 'r') as f:
            label = json.load(f)
        
        check_id = []
        for obj in label:
            try:
                if obj['obj_id'] in set(check_id):
                    continue
                #     continue
                # if obj['obj_type'] in rename.keys():
                # frame.append(i)
                # track_id.append(obj['obj_id'])
                renamed_type = rename[obj['obj_type']]
                # type_.append(renamed_type)
                # scale = obj['psr']['scale']
                # scale = f'{scale["z"]} {scale["y"]} {scale["x"]}'
                # dimensions.append(scale)
                # position = obj['psr']['position']
                # position = f'{position["x"]} {position["y"]} {position["z"]}'
                # location.append(position)
                # h_.append(obj['psr']['scale']['z'])
                # w_.append(obj['psr']['scale']['y'])
                # l_.append(obj['psr']['scale']['x'])
                # x_.append(obj['psr']['position']['x'])
                # y_.append(obj['psr']['position']['y'])
                # z_.append(obj['psr']['position']['z'])
                # rotation_y.append(obj['psr']['rotation']['z'])
                
                def roty(t, Rx=90/180*np.pi):
                    ''' Rotation about the y-axis. '''
                    c = np.cos(t)
                    s = np.sin(t)

                    X = np.array([[1, 0, 0],
                                    [0, np.cos(Rx), -np.sin(Rx)],
                                    [0, np.sin(Rx), np.cos(Rx)]])

                    # X = np.eye(3)

                    Z = np.array([[c, -s, 0],
                                    [s, c, 0],
                                    [0, 0, 1]])
                    
                    return np.matmul(Z, X)
                
                R = roty(obj['psr']['rotation']['z'])
                x = obj['psr']['position']['x']        
                y = obj['psr']['position']['y']        
                z = obj['psr']['position']['z']
                l = obj['psr']['scale']['x']
                w = obj['psr']['scale']['y']
                h = obj['psr']['scale']['z']
                
                x_corners = [l / 2, l / 2, -l / 2, -l / 2, l / 2, l / 2, -l / 2, -l / 2];
                y_corners = [h / 2, h / 2, h / 2, h / 2, -h / 2, -h / 2, -h / 2, -h / 2];
                z_corners = [w / 2, -w / 2, -w / 2, w / 2, w / 2, -w / 2, -w / 2, w / 2];
                
                corners_3d = np.dot(R, np.vstack([x_corners, y_corners, z_corners]))
                corners_3d[0, :] = corners_3d[0, :] + x  # x
                corners_3d[1, :] = corners_3d[1, :] + y  # y
                corners_3d[2, :] = corners_3d[2, :] + z  # z
                corners_3d = np.vstack([corners_3d, [1, 1, 1, 1, 1, 1, 1, 1]])
                
                point2d = np.matmul(intrinsic, np.matmul(extrinsic, corners_3d))
                pointx = np.around(point2d/point2d[2])[0]
                pointy = np.around(point2d/point2d[2])[1]
                pointx[pointx > 1920] = 1920
                pointx[pointx < 0] = 0
                pointy[pointy > 1200] = 1200
                pointy[pointy < 0] = 0
                
                # xyxy = f'{min(pointx)} {min(pointy)} {max(pointx)} {max(pointy)}'
                frame.append(i)
                track_id.append(obj['obj_id'])
                check_id.append(obj['obj_id'])
                type_.append(renamed_type)
                h_.append(obj['psr']['scale']['z'])
                w_.append(obj['psr']['scale']['y'])
                l_.append(obj['psr']['scale']['x'])
                x_.append(obj['psr']['position']['x'])
                y_.append(obj['psr']['position']['y'])
                z_.append(obj['psr']['position']['z'])
                rotation_y.append(obj['psr']['rotation']['z'])
                x1.append(min(pointx))
                y1.append(min(pointy))
                x2.append(max(pointx))
                y2.append(max(pointy))
            except:
                pass
            
    scene_df = pd.DataFrame(columns=['frame', 'track_id', 'type', 'truncated', 'occluded', 'alpha', 'x1', 'y1', 'x2', 'y2', 'h', 'w', 'l', 'x', 'y', 'z', 'rotation_y'])  
    scene_df[['frame', 'track_id', 'type', 'x1', 'y1', 'x2', 'y2', 'h', 'w', 'l', 'x', 'y', 'z', 'rotation_y']] = \
        np.asarray([frame, track_id, type_, x1, y1, x2, y2, h_, w_, l_, x_, y_, z_, rotation_y]).T
    scene_df[['truncated', 'occluded', 'alpha']] = 0
    
    scene_df.to_csv(save_path+f'{scene}.txt', index=None, header=None, sep=' ')

## # trackers
---

In [ ]:
source_path = '/data/NIA50/50-2/models/DeepFusionMOT/results/nia50_train1st/data'
save_path = '/data/NIA50/50-2/data/NIA50/train_1st/trackeval/trackers/nia50_train1st'
os.makedirs(save_path, exist_ok=True)

shutil.copytree(source_path, save_path, dirs_exist_ok=True)

# # evaluation
---

In [34]:
%cd /data/NIA50/50-2/models/TrackEval

!python scripts/run_kitti.py

/data/NIA50/50-2/models/TrackEval

Eval Config:
USE_PARALLEL         : False                         
NUM_PARALLEL_CORES   : 8                             
BREAK_ON_ERROR       : True                          
RETURN_ON_ERROR      : False                         
LOG_ON_ERROR         : /data/NIA50/50-2/models/TrackEval/error_log.txt
PRINT_RESULTS        : True                          
PRINT_ONLY_COMBINED  : False                         
PRINT_CONFIG         : True                          
TIME_PROGRESS        : True                          
DISPLAY_LESS_PROGRESS : False                         
OUTPUT_SUMMARY       : True                          
OUTPUT_EMPTY_CLASSES : True                          
OUTPUT_DETAILED      : True                          
PLOT_CURVES          : True                          

Kitti2DBox Config:
PRINT_CONFIG         : True                          
GT_FOLDER            : /data/NIA50/50-2/data/NIA50/train_1st/trackeval/gt
TRACKERS_FOLDER      : /data/N

In [ ]:
%cd /data/NIA50/50-2/models/TrackEval

# USE_PARALLEL = True
# NUM_PARALLEL_CORES = 4
# # CLASSES_TO_EVAL = ['Car', 'Pedestrian', 'Cyclist']
# GT_FOLDER = '/data/NIA50/50-2/data/NIA50/train_1st/trackeval/gt'
# TRACKERS_FOLDER = '/data/NIA50/50-2/data/NIA50/train_1st/trackeval/trackers'
# # TRACKER_SUB_FOLDER = 'trackers'
# OUTPUT_FOLDER = '/data/NIA50/50-2/models/TrackEval/results/nia50_train1st'

!python scripts/run_kitti.py

In [ ]:
%cd /data/NIA50/50-2/models/TrackEval

!python scripts/run_kitti.py --help